## Vivino Webscraper 

#### Introduction: 
Vivino is a popular wine enthusiast website database of information about wine from around the world. This project is a proof of concept for scraping data from Vivino for the purposes of wine understanding the wine market in Spain. This proof of concept has some limitations. First, the list of wines scraped are not a complete and comprehensive view of Spanish wine, as wine culture in Spain has traditionally been that of small production and barrel sales. For this reason, many small wine producers are not included in Vivinos database and therefore not represented in the data. Second, the wines shown only include wines currently available for purchase. Lastly, as a consumer database, the results of ratings may not represent professional opinions and may be limited by the 5-star rating system due to its overly simplistic nature. 

url = https://www.vivino.com/api/explore/explore?country_code=kr&currency_code=KRW&grape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&price_range_max=500000&price_range_min=0&page=1&language=en

In [12]:
# with open('response_data.json', 'r', encoding='utf-8') as file:
#     data = json.load(file)
# n_matches = data['explore_vintage']['records_matched']
# matches = data['explore_vintage']['matches']
# 
# print(n_matches)
# print(matches)

3660
[{'vintage': {'id': 177404982, 'seo_name': 'weingut-carl-loewen-eighteen-ninetysix-riesling-2023', 'name': 'Carl Loewen 1896 Riesling 2023', 'statistics': {'status': 'Normal', 'ratings_count': 115, 'ratings_average': 5, 'labels_count': 15, 'wine_ratings_count': 354, 'wine_ratings_average': 4.6, 'wine_status': ''}, 'image': {'location': '//images.vivino.com/thumbs/Q8yn9Dn7SZa_V_IBeD51yw_pl_480x640.png', 'variations': {'bottle_large': '//images.vivino.com/thumbs/Q8yn9Dn7SZa_V_IBeD51yw_pb_x960.png', 'bottle_medium': '//images.vivino.com/thumbs/Q8yn9Dn7SZa_V_IBeD51yw_pb_x600.png', 'bottle_medium_square': '//images.vivino.com/thumbs/Q8yn9Dn7SZa_V_IBeD51yw_pb_600x600.png', 'bottle_small': '//images.vivino.com/thumbs/Q8yn9Dn7SZa_V_IBeD51yw_pb_x300.png', 'bottle_small_square': '//images.vivino.com/thumbs/Q8yn9Dn7SZa_V_IBeD51yw_pb_300x300.png', 'label': '//images.vivino.com/thumbs/Q8yn9Dn7SZa_V_IBeD51yw_pl_480x640.png', 'label_large': '//images.vivino.com/thumbs/Q8yn9Dn7SZa_V_IBeD51yw_pl_3

In [16]:
import requests
import json
import pandas as pd
import numpy as np
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    # "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
}

payload = {
        "country_codes": "kr",  
        "currency_code": "KRW",
        # "food_ids[]": [4],
        "grape_filter": "varietal",
        "min_rating": "1",
        "order_by": "ratings_average",
        "order": "desc",
        "page": 1,
        "price_range_max": "500000",
        "price_range_min": "0",
        "language": "en",
}

 
response = requests.get('https://www.vivino.com/api/explore/explore?',
                 params=payload, headers=headers)
data = response.json()
n_matches = data['explore_vintage']['records_matched']

In [3]:
column_names=["vintage_id", "vintage_name", "vintage_year", "vintage_price", "vintage_ratings_average", "vintage_ratings_count", "vintage_wine_id", "vintage_wine_name",  "vintage_winery", "vintage_country", "vintage_region", "vintage_wine_type_id", "acidity", "fizziness", "intensity", "sweetness", "tannin", "flavor","varietal_name","body_description", "acidity_description", "foods" ]

In [1]:
# 루프 밖에서 빈 리스트 생성
all_results = []
# int(n_matches / 25) + 1
for i in range(2):
    # 각 페이지당 25개의 result 존재, 페이지 넘길 때마다 알림 호출
    # payload['page'] = i + 1
    payload = {
        "country_codes": "kr",  
        "currency_code": "KRW",
        # "food_ids[]": [4],
        "grape_filter": "varietal",
        "min_rating": "1",
        "order_by": "ratings_average",
        "order": "desc",
        "page": str(i + 1),
        "price_range_max": "500000",
        "price_range_min": "0",
        "language": "en",
    }
    
    print(f'Requesting data from page: {payload["page"]}')
    response_page = requests.get('https://www.vivino.com/api/explore/explore', params=payload, headers=headers)
    print(payload)
    if response_page.status_code == 200:
        # JSON 데이터를 딕셔너리 형태로 파싱
        data_page = response_page.json()
        print(f"Page {i + 1}: {len(data_page['explore_vintage']['matches'])} results")
        # JSON 데이터를 파일로 저장
        with open(f'response_page{i+1}.json', 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

        print("JSON 데이터가 파일로 저장되었습니다.")

        # for match in data_page['explore_vintage']['matches']:
        #     # 필요한 데이터 추출
        #     vintage_id = match['vintage']['id']
        #     vintage_name = match['vintage']['name']
        #     vintage_year = match["vintage"]["year"]
        #     vintage_price = match["prices"][0]["amount"]
        #     vintage_ratings_average = match["vintage"]["statistics"]["ratings_average"]
        #     vintage_ratings_count = match["vintage"]["statistics"]["ratings_count"]
        # 
        #     if match['vintage']['wine'] is not None:
        #         vintage_wine_id = match["vintage"]["wine"]["id"]
        #         vintage_wine_name = f'{match["vintage"]["wine"]["name"]} {match["vintage"]["year"]}'
        #         vintage_winery = match["vintage"]["wine"]["winery"]["name"]
        #         vintage_country = match['vintage']['wine']['region']['country']['name']
        #         vintage_region = match['vintage']['wine']['region']['name']
        #         vintage_wine_type_id = match['vintage']['wine']['type_id']
        # 
        #         # 맛과 관련된 정보 초기화
        #         taste = match['vintage']['wine']['taste']
        #         acidity = np.nan
        #         fizziness = np.nan
        #         intensity = np.nan
        #         sweetness = np.nan
        #         tannin = np.nan
        #         flavor = np.nan
        # 
        #         if taste is not None:
        #             structure = taste['structure']
        #             if structure is not None:
        #                 acidity = structure.get('acidity', np.nan)
        #                 fizziness = structure.get('fizziness', np.nan)
        #                 intensity = structure.get('intensity', np.nan)
        #                 sweetness = structure.get('sweetness', np.nan)
        #                 tannin = structure.get('tannin', np.nan)
        # 
        #             # flavor가 있으면 첫 번째 요소를 가져옴
        #             flavor_list = taste['flavor']
        #             if len(flavor_list) > 0:
        #                 flavor = flavor_list[0].get('group', np.nan)
        # 
        #         # 음식 페어링 관련 정보
        #         style = match['vintage']['wine']['style']
        #         if style is not None:
        #             varietal_name = style.get('varietal_name', np.nan)
        #             body_description = style.get('body_description', np.nan)
        #             acidity_description = style.get('acidity_description', np.nan)
        #         foods = []
        #         if style and style.get('food'):
        #             foods = [food['name'] for food in style['food']]
        # 
        #         # 수집한 데이터를 리스트에 추가
        #         all_results.append((vintage_id, vintage_name, vintage_year, vintage_price, vintage_ratings_average, 
        #                             vintage_ratings_count, vintage_wine_id, vintage_wine_name, vintage_winery, 
        #                             vintage_country, vintage_region, vintage_wine_type_id, acidity, fizziness, 
        #                             intensity, sweetness, tannin, flavor,varietal_name,body_description, acidity_description, foods))
    else:
        print("데이터를 가져오는 데 실패했습니다. 상태 코드:", response_page.status_code)

# #최종 DataFrame 생성
# vivino_df = pd.DataFrame(all_results, columns=column_names)


Requesting data from page: 1


NameError: name 'requests' is not defined

In [49]:
vivino_df.shape

(3675, 22)

In [50]:
vivino_df.head(10)

,vintage_id,vintage_name,vintage_year,vintage_price,vintage_ratings_average,vintage_ratings_count,vintage_wine_id,vintage_wine_name,vintage_winery,vintage_country,...,acidity,fizziness,intensity,sweetness,tannin,flavor,varietal_name,body_description,acidity_description,foods
0,177404982,Carl Loewen 1896 Riesling 2023,2023,82045,5.0,115,1945087,1896 Riesling 2023,Carl Loewen,Germany,...,4.61,NaN,3.32,2.05,NaN,tree_fruit,Riesling,Light-bodied,High,"[Pork, Shellfish, Spicy food, Poultry, Cured M..."
1,1510217,Château Haut-Brion Pessac-Léognan (Premier Gra...,1989,3244496,4.8,1538,1152755,Pessac-Léognan (Premier Grand Cru Classé) 1989,Château Haut-Brion,France,...,4.22,NaN,4.08,1.67,4.23,oak,Pessac-Léognan,Full-bodied,High,"[Beef, Lamb, Game (deer, venison), Poultry]"
2,2611979,Château Latour Grand Vin Pauillac (Premier Gra...,1982,2759686,4.8,1445,1655970,Grand Vin Pauillac (Premier Grand Cru Classé) ...,Château Latour,France,...,4.12,NaN,4.12,1.68,4.13,oak,Pauillac,Full-bodied,High,"[Beef, Lamb, Game (deer, venison), Poultry]"
3,170413977,Domaines Ott Clos Mireille Rosé (Coeur de Grai...,2022,59594,4.8,581,1382222,Clos Mireille Rosé (Coeur de Grain) 2022,Domaines Ott,France,...,3.77,NaN,2.50,1.33,NaN,citrus_fruit,Rosé,Medium-bodied,High,"[Pork, Shellfish, Vegetarian, Poultry]"
4,1688597,Krug Clos du Mesnil 2000,2000,2175423,4.8,347,79635,Clos du Mesnil 2000,Krug,France,...,4.54,4.22,3.97,NaN,NaN,non_oak,Champagne,Medium-bodied,High,"[Pork, Rich fish (salmon, tuna etc), Shellfish..."
5,174958238,Jakob Schneider Niederhäuser Felsensteyer Ries...,2022,23793,4.8,123,3756275,Niederhäuser Felsensteyer Riesling Trocken 2022,Jakob Schneider,Germany,...,4.66,NaN,2.90,1.59,NaN,earth,Riesling,Light-bodied,High,"[Pork, Shellfish, Spicy food, Poultry, Cured M..."
6,1327613,Krohn Colheita Port 1966,1966,447517,4.8,91,1136061,Colheita Port 1966,Krohn,Portugal,...,3.14,NaN,4.39,4.66,NaN,non_oak,Colheita Port,Very full-bodied,Medium,"[Sweet desserts, Mature and hard cheese, Blue ..."
7,1429443,Château d'Yquem Sauternes 1959,1959,2349462,4.8,88,1153863,Sauternes 1959,Château d'Yquem,France,...,4.32,NaN,4.67,4.87,NaN,earth,Sauternes,Full-bodied,High,"[Fruity desserts, Blue cheese]"
8,1331826,E. Guigal Côte-Rôtie La Landonne 1990,1990,1485502,4.8,69,36404,Côte-Rôtie La Landonne 1990,E. Guigal,France,...,3.90,NaN,4.27,1.70,4.13,oak,Côte-Rotie,Very full-bodied,High,"[Beef, Lamb, Game (deer, venison), Poultry]"
9,3556730,Royal Tokaji Essencia 2000,2000,948775,4.8,45,1140747,Essencia 2000,Royal Tokaji,Hungary,...,NaN,NaN,NaN,NaN,NaN,earth,Côte-Rotie,Very full-bodied,High,[]


In [51]:
vivino_df.tail(10)

,vintage_id,vintage_name,vintage_year,vintage_price,vintage_ratings_average,vintage_ratings_count,vintage_wine_id,vintage_wine_name,vintage_winery,vintage_country,...,acidity,fizziness,intensity,sweetness,tannin,flavor,varietal_name,body_description,acidity_description,foods
3665,31027695,Antinori Pian delle Vigne Rosso di Montalcino ...,2015,30610,3.9,2252,2685679,Pian delle Vigne Rosso di Montalcino 2015,Antinori,Italy,...,3.12,NaN,3.16,1.64,2.95,red_fruit,Red,Medium-bodied,Medium,"[Beef, Lamb, Veal, Poultry, Cured Meat]"
3666,3253022,Monte Rossa Coupé,N.V.,47258,3.9,2246,1777850,Coupé N.V.,Monte Rossa,Italy,...,3.62,3.31,3.05,NaN,NaN,microbio,Sparkling,Medium-bodied,High,"[Shellfish, Appetizers and snacks, Lean fish, ..."
3667,1645481,Rock Bare Shiraz 2003,2003,45282,3.9,2221,1105827,Shiraz 2003,Rock Bare,Australia,...,3.07,NaN,4.81,2.10,3.19,oak,Shiraz,Full-bodied,Medium,"[Beef, Lamb, Game (deer, venison)]"
3668,3057393,Brokenwood Shiraz 2004,2004,56064,3.9,2154,31424,Shiraz 2004,Brokenwood,Australia,...,3.01,NaN,4.60,2.10,3.04,black_fruit,Shiraz,Full-bodied,Medium,"[Beef, Lamb, Game (deer, venison)]"
3669,167081784,Château Beauchene Côtes du Rhône Premier Terro...,2021,18647,3.9,2121,1136434,Côtes du Rhône Premier Terroir 2021,Château Beauchene,France,...,3.39,NaN,3.93,1.67,3.21,black_fruit,Red,Full-bodied,High,"[Beef, Lamb, Game (deer, venison)]"
3670,162907519,Masciarelli Montepulciano d'Abruzzo 2020,2020,15365,3.9,2088,93061,Montepulciano d'Abruzzo 2020,Masciarelli,Italy,...,3.92,NaN,2.71,1.36,2.97,red_fruit,Montepulciano d'Abruzzo,Medium-bodied,High,"[Beef, Pasta, Veal, Pork]"
3671,164818931,Château Olivier Pessac-Léognan Blanc (Grand Cr...,1964,316991,3.9,2033,87425,Pessac-Léognan Blanc (Grand Cru Classé de Grav...,Château Olivier,France,...,3.17,NaN,3.80,1.96,NaN,citrus_fruit,White,Medium-bodied,High,"[Pork, Vegetarian, Poultry, Appetizers and sna..."
3672,23213221,Costaripa Mattia Vezzola Brut,N.V.,37964,3.9,2006,3429423,Mattia Vezzola Brut N.V.,Costaripa,Italy,...,3.29,3.82,2.89,NaN,NaN,tree_fruit,Sparkling,Medium-bodied,Medium,"[Appetizers and snacks, Aperitif]"
3673,169670734,Pegasus Bay Sauvignon - Sémillon 2021,2021,23017,3.9,1905,16114,Sauvignon - Sémillon 2021,Pegasus Bay,New Zealand,...,NaN,NaN,NaN,NaN,NaN,citrus_fruit,Sparkling,Medium-bodied,Medium,[]
3674,4046901,Abbazia di S. Gaudenzio Moscato Dolce,N.V.,25876,3.9,1822,2115561,Moscato Dolce N.V.,Abbazia di S. Gaudenzio,Italy,...,3.01,3.94,3.02,NaN,NaN,tree_fruit,Sparkling,Medium-bodied,Medium,"[Appetizers and snacks, Aperitif]"


In [52]:
vivino_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3675 entries, 0 to 3674
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   vintage_id               3675 non-null   int64  
 1   vintage_name             3675 non-null   object 
 2   vintage_year             3675 non-null   object 
 3   vintage_price            3675 non-null   int64  
 4   vintage_ratings_average  3675 non-null   float64
 5   vintage_ratings_count    3675 non-null   int64  
 6   vintage_wine_id          3675 non-null   int64  
 7   vintage_wine_name        3675 non-null   object 
 8   vintage_winery           3675 non-null   object 
 9   vintage_country          3675 non-null   object 
 10  vintage_region           3675 non-null   object 
 11  vintage_wine_type_id     3675 non-null   int64  
 12  acidity                  3555 non-null   float64
 13  fizziness                507 non-null    float64
 14  intensity               

In [53]:
# DataFrame을 CSV 파일로 저장
vivino_df.to_csv('vivino_data_241107.csv', index=False, encoding='utf-8-sig')
print("DataFrame이 vivino_data.csv로 저장되었습니다.")


DataFrame이 vivino_data.csv로 저장되었습니다.


In [114]:
# import numpy as np
# 
# all_results = []
# for i in range(int(n_matches / 25)):
#     # 각 페이지당 25개의 result 존재, 페이지 넘길 때마다 알림 호출
#     payload['page'] = i + 1
#     print(f'Requesting data from page: {payload["page"]}')
# 
#     r = requests.get('https://www.vivino.com/api/explore/explore?',
#                  params=payload, headers=headers)
#     results = [
#         (
#             t["vintage"]["year"],
#             t["vintage"]["wine"]["id"],
#             f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
#             t["vintage"]["statistics"]["ratings_average"],
#             t["vintage"]["statistics"]["ratings_count"],
#             t["prices"][0]["amount"],
#             t["vintage"]["wine"]["winery"]["name"],
#             t['vintage']['wine']['region']['country']['name'],
#             t['vintage']['wine']['region']['name'],
#             t['vintage']['wine']['type_id'],
#             # Safely accessing and returning np.nan if any part is None
#             (t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure", {}).get('acidity', np.nan) 
#              if t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure") is not None else np.nan),           
#             (t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure", {}).get('fizziness', np.nan) 
#              if t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure") is not None else np.nan),            
#             (t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure", {}).get('intensity', np.nan) 
#              if t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure") is not None else np.nan),            
#             (t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure", {}).get('sweetness', np.nan) 
#              if t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure") is not None else np.nan),            
#             (t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure", {}).get('tannin', np.nan) 
#              if t.get("vintage", {}).get("wine", {}).get("taste", {}).get("structure") is not None else np.nan),           
#             t.get("vintage", {}).get("wine", {}).get("taste", {}).get("flavor", [{}])[0].get("group", np.nan) 
#             if t.get("vintage", {}).get("wine", {}).get("taste", {}).get("flavor") else np.nan,
#             
#         )
#         for t in r.json()["explore_vintage"]["matches"] 
#     ]   
#     
#     all_results.extend(results)
# 
# df = pd.DataFrame(all_results, columns=column_names)
    

In [32]:

# for i in range(int(n_matches / 25)):
#     # 각 페이지당 25개의 result 존재, 페이지 넘길 때마다 알림 호출
#     payload['page'] = i + 1
#     print(f'Requesting data from page: {payload["page"]}')
#     matches = data['explore_vintage']['matches'][payload["page"]]
    # print(matches)
    # r = requests.get('https://www.vivino.com/api/explore/explore?',
    #              params=payload, headers=headers)
# all_results = []
# for i in range(int(n_matches / 25)):
#     # 각 페이지당 25개의 result 존재, 페이지 넘길 때마다 알림 호출
#     payload['page'] = i + 1
#     print(f'Requesting data from page: {payload["page"]}')
#     response_page = requests.get('https://www.vivino.com/api/explore/explore?',
#                  params=payload, headers=headers)
# 
#     if response_page.status_code == 200:
#         # JSON 데이터를 딕셔너리 형태로 파싱
#         data_page = response_page.json()
#         results = []
#         for match in data_page['explore_vintage']['matches']:
#         # match = data_page['explore_vintage']['matches'][payload["page"]]
#         # print(match)
#         # # JSON 데이터를 파일로 저장
#         # with open(f'response_page{i+1}.json', 'w', encoding='utf-8') as file:
#         #     json.dump(data, file, ensure_ascii=False, indent=4)
#         # 
#         # print("JSON 데이터가 파일로 저장되었습니다.")
#             
#             vintage_id = match['vintage']['id']
#             vintage_name = match['vintage']['name']
#             vintage_year = match["vintage"]["year"]
#             vintage_price = match["prices"][0]["amount"]
#             vintage_ratings_average = match["vintage"]["statistics"]["ratings_average"]
#             vintage_ratings_count = match["vintage"]["statistics"]["ratings_count"]
#             
#             if match['vintage']['wine'] is None:
#                 print('wine이 존재하지 않습니다.')
#             else:
#                 vintage_wine_id = match["vintage"]["wine"]["id"]
#                 vintage_wine_name = f'{match["vintage"]["wine"]["name"]} {match["vintage"]["year"]}'
#                 vintage_winery = match["vintage"]["wine"]["winery"]["name"]
#                 vintage_country = match['vintage']['wine']['region']['country']['name']
#                 vintage_region = match['vintage']['wine']['region']['name']
#                 vintage_wine_type_id = match['vintage']['wine']['type_id']
#                 
#                 taste = match['vintage']['wine']['taste']
#                 acidity = np.nan
#                 fizziness = np.nan
#                 intensity = np.nan
#                 sweetness = np.nan
#                 tannin = np.nan
#                 flavor = np.nan
#                 
#                 if taste is None:
#                     pass
#                 else:
#                     structure = taste['structure']
#                     if structure is None:
#                         pass
#                     else:
#                         acidity = structure['acidity']
#                         fizziness = structure['fizziness']
#                         intensity = structure['intensity']
#                         sweetness = structure['sweetness']
#                         tannin = structure['tannin']
#                         if acidity is None:
#                             pass
#                         if fizziness is None:
#                             pass
#                         if intensity is None:
#                             pass
#                         if sweetness is None:
#                             pass
#                         if tannin is None:
#                             pass
#                     flavor_list = taste['flavor']
#                     if len(flavor_list) == 0:
#                         pass
#                     else:
#                         flavor = flavor_list[0]['group']
#                         if flavor is None:
#                             pass
#             
#                 style = match['vintage']['wine']['style']
#                 foods = [] # 초기값
#                 if style is None:
#                     pass
#                 else:
#                     foods_list = style['food']
#                     if foods_list is None:
#                         pass
#                     else:
#                         for food in foods_list:
#                             foods.append(food['name'])
#                     
#                 results.append((vintage_id, vintage_name, vintage_year, vintage_price, vintage_ratings_average, vintage_ratings_count, vintage_wine_id, vintage_wine_name,  vintage_winery, vintage_country, vintage_region, vintage_wine_type_id, acidity, fizziness, intensity, sweetness, tannin, flavor, foods))
#         df = pd.DataFrame(results, columns=column_names)
#     else:
#         print("데이터를 가져오는 데 실패했습니다. 상태 코드:", response.status_code)
    
    


Requesting data from page: 1
Requesting data from page: 2
Requesting data from page: 3
Requesting data from page: 4
Requesting data from page: 5
Requesting data from page: 6
Requesting data from page: 7
Requesting data from page: 8
Requesting data from page: 9
Requesting data from page: 10
Requesting data from page: 11
Requesting data from page: 12
Requesting data from page: 13
Requesting data from page: 14
Requesting data from page: 15
Requesting data from page: 16
Requesting data from page: 17
Requesting data from page: 18
Requesting data from page: 19
Requesting data from page: 20
Requesting data from page: 21
Requesting data from page: 22
Requesting data from page: 23
Requesting data from page: 24
Requesting data from page: 25
Requesting data from page: 26
Requesting data from page: 27
Requesting data from page: 28
Requesting data from page: 29
Requesting data from page: 30
Requesting data from page: 31
Requesting data from page: 32
Requesting data from page: 33
Requesting data fro